# Computer Vision Project 2

# Part I

## Import necesasry libraries

In [1]:
import tensorflow as tf
import numpy as np
import os
from matplotlib import pyplot as plt
import random
random.seed(0)
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.patches as patches
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input
from PIL import Image
import csv
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Reshape


### Load the data

In [40]:
load_array = np.load('D:\AIML- UT\Vision systems\Project 2\Part 1- Train data - images.npy', allow_pickle=True)

In [41]:
load_array[0][0]

array([[[42, 37, 34],
        [56, 51, 48],
        [71, 66, 63],
        ...,
        [23, 33, 34],
        [26, 36, 37],
        [28, 38, 39]],

       [[40, 35, 32],
        [51, 46, 43],
        [64, 59, 56],
        ...,
        [27, 36, 35],
        [24, 33, 32],
        [26, 35, 34]],

       [[43, 38, 35],
        [51, 46, 43],
        [61, 56, 53],
        ...,
        [28, 30, 27],
        [33, 35, 32],
        [35, 37, 34]],

       ...,

       [[56, 47, 40],
        [57, 48, 41],
        [61, 52, 45],
        ...,
        [67, 48, 42],
        [55, 35, 28],
        [60, 40, 33]],

       [[53, 44, 37],
        [54, 45, 38],
        [57, 48, 41],
        ...,
        [59, 40, 34],
        [60, 40, 33],
        [54, 34, 27]],

       [[53, 44, 37],
        [54, 45, 38],
        [57, 48, 41],
        ...,
        [59, 40, 34],
        [70, 50, 43],
        [64, 44, 37]]], dtype=uint8)

In [42]:
load_array[0][1]

[{'label': ['Face'],
  'notes': '',
  'points': [{'x': 0.08615384615384615, 'y': 0.3063063063063063},
   {'x': 0.1723076923076923, 'y': 0.45345345345345345}],
  'imageWidth': 650,
  'imageHeight': 333},
 {'label': ['Face'],
  'notes': '',
  'points': [{'x': 0.583076923076923, 'y': 0.2912912912912913},
   {'x': 0.6584615384615384, 'y': 0.46846846846846846}],
  'imageWidth': 650,
  'imageHeight': 333}]

## Data preparation

### Convert npy data to images and save them

In [5]:
for i in range(409):
    a=load_array[i][0]
    image = Image.fromarray(a)
    image.save('D:\AIML- UT\Vision systems\Project 2\images\image-'+str(i)+'.png')
    

KeyboardInterrupt: 

### Extract data

#### From the npy data we extract data like path, height, width,x0,y0,x1,y1, and label of each image and converting them into a csv file

In [43]:
path=[]
x0=[]
y0=[]
label=[]
x1=[]
y1=[]
width=[]
height=[]

for i in range(409):
    l=load_array[i][1]
    for j in range(len(l)):
        filepath='D:/AIML- UT/Vision systems/Project 2/images/image-'+str(i)+'.png'
        path.append(filepath)
        label.append(l[j]['label'][0])
        x0.append(l[j]['points'][0]['x'] * l[j]['imageWidth'] )
        y0.append(l[j]['points'][0]['y'] * l[j]['imageHeight'])
        x1.append(l[j]['points'][1]['x'] * l[j]['imageWidth'])
        y1.append(l[j]['points'][1]['y'] * l[j]['imageHeight'])
        width.append(l[j]['imageWidth'])
        height.append(l[j]['imageHeight'])

In [44]:
x=path
path = pd.Series(path)
label = pd.Series(label)
x0_series = pd.Series(x0)
y0_series = pd.Series(y0)
x1_series = pd.Series(x1)
y1_series = pd.Series(y1)
width=pd.Series(width)
height=pd.Series(height)


In [45]:
df = pd.concat([path, height,width,x0_series,y0_series,x1_series,y1_series,label], axis=1)


In [46]:
df.head()

,0,1,2,3,4,5,6,7
0,D:/AIML- UT/Vision systems/Project 2/images/im...,333,650,56.000000,102.000000,112.000000,151.000000,Face
1,D:/AIML- UT/Vision systems/Project 2/images/im...,333,650,379.000000,97.000000,428.000000,156.000000,Face
2,D:/AIML- UT/Vision systems/Project 2/images/im...,697,1280,902.795233,162.125249,984.615385,252.194831,Face
3,D:/AIML- UT/Vision systems/Project 2/images/im...,240,460,216.000000,12.000000,279.000000,80.000000,Face
4,D:/AIML- UT/Vision systems/Project 2/images/im...,240,460,289.000000,2.000000,343.000000,74.000000,Face


In [47]:
df.to_csv('train.csv',header=None,index=False)

### Create the labels (Bounding box)

In [48]:
y=[]
for i in range(len(x)):
        n=[]
        n.append(x0[i])
        n.append(y0[i])
        n.append(x1[i])
        n.append(y1[i])
        y.append(n)

In [49]:
len(y)

1132

In [50]:
len(x)

1132

In [51]:
filename = x[2]
unscaled = cv2.imread(filename,1)
region = y[2]
print(y[2])

color = (255, 0, 0)
thickness = 2
start_point = (int(region[0]), int(region[1]))
end_point = (int(region[2]), int(region[3]))

image= cv2.rectangle(unscaled, start_point, end_point, color, thickness)
cv2.imshow('image', image)
cv2.waitKey(0) 
cv2.destroyAllWindows() 



[902.7952329360779, 162.12524850894633, 984.6153846153846, 252.1948310139165]


In [52]:
x_train=x[:1100]
x_val=x[1100:]
y_train=y[:1100]
y_val=y[1100:]

### Create train data

In [53]:
IMAGE_SIZE=128
for i, f in enumerate(x_train):
  img = Image.open(f) # Read image
  img = img.resize((IMAGE_SIZE, IMAGE_SIZE)) # Resize image
  img = img.convert('RGB')

  x_train[i] = preprocess_input(np.array(img, dtype=np.float32)) # Convert to float32 array
  img.close()

In [54]:
x_train = np.array(x_train)

In [55]:
x_train.shape

(1100, 128, 128, 3)

In [56]:
y_train=np.array(y_train)

In [57]:
y_train.shape

(1100, 4)

### Create test data

In [58]:
for i, f in enumerate(x_val):
  img = Image.open(f) # Read image
  img = img.resize((IMAGE_SIZE, IMAGE_SIZE)) # Resize image
  img = img.convert('RGB')

  x_val[i] = preprocess_input(np.array(img, dtype=np.float32)) # Convert to float32 array
  img.close()

In [59]:
x_val=np.array(x_val)
y_val=np.array(y_val)

In [60]:
x_val.shape

(32, 128, 128, 3)

In [61]:
y_val.shape

(32, 4)

## Model Building and training

### Using Mobile net for face detection

In [66]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Reshape

ALPHA = 1.0 # Width hyper parameter for MobileNet (0.25, 0.5, 0.75, 1.0). Higher width means more accurate but slower

def create_model(trainable=True):
    model = MobileNet(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, alpha=ALPHA) # Load pre-trained mobilenet
    # Do not include classification (top) layer

    # to freeze layers, except the new top layer, of course, which will be added below
    for layer in model.layers:
        layer.trainable = trainable

    # Add new top layer which is a conv layer of the same size as the previous layer so that only 4 coords of BBox can be output
    x0 = model.layers[-1].output
    x1 = Conv2D(4, kernel_size=4, name="coords")(x0)
    # In the line above kernel size should be 3 for img size 96, 4 for img size 128, 5 for img size 160 etc.
    x2 = Reshape((4,))(x1) # These are the 4 predicted coordinates of one BBox

    return Model(inputs=model.input, outputs=x2)

### Using IOU as metric 

In [67]:
def IOU(y_true, y_pred):
    intersections = 0
    unions = 0
    # set the types so we are sure what type we are using

    gt = y_true
    pred = y_pred
    # Compute interection of predicted (pred) and ground truth (gt) bounding boxes
    diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
    diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
    intersection = diff_width * diff_height

    # Compute union
    area_gt = gt[:,2] * gt[:,3]
    area_pred = pred[:,2] * pred[:,3]
    union = area_gt + area_pred - intersection

    # Compute intersection and union over multiple boxes
    for j, _ in enumerate(union):
      if union[j] > 0 and intersection[j] > 0 and union[j] >= intersection[j]:
        intersections += intersection[j]
        unions += union[j]

    # Compute IOU. Use epsilon to prevent division by zero
    iou = np.round(intersections / (unions + tf.keras.backend.epsilon()), 4)
    # This must match the type used in py_func
    iou = iou.astype(np.float32)
    return iou

In [68]:
def IoU(y_true, y_pred):
    iou = tf.py_function(IOU, [y_true, y_pred], Tout=tf.float32)
    return iou

In [69]:
model = create_model(True) # Arg is False, if you want to freeze lower layers for fast training (but low accuracy)
model.summary() # Print summary

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128   

In [70]:
# Compile the model
model.compile(loss="mean_squared_error", optimizer="adam", metrics=[IoU]) # Regression loss is MSE

In [71]:
# Use earlystopping
#callback = tf.keras.callbacks.EarlyStopping(monitor='val_IoU', patience=5, min_delta=0.01)

# Fit the model
#model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20, batch_size=32, callbacks=[callback])
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20, batch_size=32)

Train on 1100 samples, validate on 32 samples
Epoch 1/20
1100/1100 [==============================] - 66s 60ms/sample - loss: 242971.8791 - IoU: 0.2237 - val_loss: 122098.2344 - val_IoU: 0.1497
Epoch 2/20
1100/1100 [==============================] - 59s 54ms/sample - loss: 125202.4705 - IoU: 0.1761 - val_loss: 66764.8906 - val_IoU: 0.1890
Epoch 3/20
1100/1100 [==============================] - 65s 59ms/sample - loss: 97101.0752 - IoU: 0.1926 - val_loss: 64731.0859 - val_IoU: 0.2204
Epoch 4/20
1100/1100 [==============================] - 68s 62ms/sample - loss: 103283.5680 - IoU: 0.1868 - val_loss: 181623.2812 - val_IoU: 0.1163
Epoch 5/20
1100/1100 [==============================] - 62s 56ms/sample - loss: 94968.4348 - IoU: 0.1827 - val_loss: 56795.8750 - val_IoU: 0.2305
Epoch 6/20
1100/1100 [==============================] - 60s 54ms/sample - loss: 82130.6330 - IoU: 0.2339 - val_loss: 71012.1562 - val_IoU: 0.1666
Epoch 7/20
1100/1100 [==============================] - 62s 56ms/sample -

## Evaluation

### Final model accuracy

In [72]:
model.evaluate(x_val, y_val)

32/32 [==============================] - 0s 10ms/sample - loss: 73316.1875 - IoU: 0.2186


[73316.1875, 0.2186]

### Save model

In [77]:
model.save('Face_detection')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: Face_detection\assets


### Evaluate with test data

In [73]:
filename = 'D:AIML- UT/Vision systems/Project 2/Part 1Test Data - Prediction Image.jpeg'
unscaled = cv2.imread(filename)

In [74]:
IMAGE_SIZE=128
image_height, image_width, _ = unscaled.shape
image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE)) # Rescaled image to run the network
feat_scaled = preprocess_input(np.array(image, dtype=np.float32))
print ("Size of original input: ", image.shape)
print("-------------------------------")
print("Size of scaled input: ", feat_scaled.shape)

Size of original input:  (128, 128, 3)
-------------------------------
Size of scaled input:  (128, 128, 3)


In [75]:
region = model.predict(x=np.array([feat_scaled]))[0]

In [76]:
color = (255, 0, 0)
thickness = 2
start_point = (int(region[0]), int(region[1]))
end_point = (int(region[2]), int(region[3]))

image= cv2.rectangle(unscaled, start_point, end_point, color, thickness)
cv2.imshow('image', image)
cv2.waitKey(0) 
cv2.destroyAllWindows() 

### Conclusion: Poor results, Training with more images can improve accuracy. Training for few more epochs might improve the results slightly.

# Part 2

## Load dataset

In [2]:
 import os
 labels = os.listdir('D:/AIML- UT/Vision systems/Project 2/PINS')


In [3]:

img_size = 224
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img),1) #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [4]:
train = get_data('D:/AIML- UT/Vision systems/Project 2/PINS')

In [5]:

cv2.imshow('image', train[0][0])
cv2.waitKey(0) 
cv2.destroyAllWindows() 

In [6]:
print(labels[train[0][1]])

pins_Aaron Paul


## Data preparation

In [7]:
x_train = []
y_train = []
for feature, label in train:
  x_train.append(feature)
  y_train.append(label)

In [8]:
print("Total no of images :",len(x_train))
print("Total no of labels :",len(y_train))

Total no of images : 10770
Total no of labels : 10770


In [9]:
x_train = np.array(x_train)
y_train= np.array(y_train)

In [10]:
print("Shape of data :",x_train.shape)
print("Shape of labels :",y_train.shape)

Shape of data : (10770, 224, 224, 3)
Shape of labels : (10770,)


In [11]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout,  Activation
from keras.models import Sequential


Using TensorFlow backend.


In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.30, random_state=1)

### Data augmentation

In [70]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [71]:
val_gen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)

val_gen.fit(X_test)

## Model Building and training

In [13]:
def vgg_face(weights_path=None):
    img = Input(shape=(224, 224,3))

    pad1_1 = ZeroPadding2D(padding=(1, 1))(img)
    conv1_1 = Convolution2D(64, 3, 3, activation='relu', name='conv1_1')(pad1_1)
    pad1_2 = ZeroPadding2D(padding=(1, 1))(conv1_1)
    conv1_2 = Convolution2D(64, 3, 3, activation='relu', name='conv1_2')(pad1_2)
    pool1 = MaxPooling2D((2, 2), strides=(2, 2))(conv1_2)

    pad2_1 = ZeroPadding2D((1, 1))(pool1)
    conv2_1 = Convolution2D(128, 3, 3, activation='relu', name='conv2_1')(pad2_1)
    pad2_2 = ZeroPadding2D((1, 1))(conv2_1)
    conv2_2 = Convolution2D(128, 3, 3, activation='relu', name='conv2_2')(pad2_2)
    pool2 = MaxPooling2D((2, 2), strides=(2, 2))(conv2_2)

    pad3_1 = ZeroPadding2D((1, 1))(pool2)
    conv3_1 = Convolution2D(256, 3, 3, activation='relu', name='conv3_1')(pad3_1)
    pad3_2 = ZeroPadding2D((1, 1))(conv3_1)
    conv3_2 = Convolution2D(256, 3, 3, activation='relu', name='conv3_2')(pad3_2)
    pad3_3 = ZeroPadding2D((1, 1))(conv3_2)
    conv3_3 = Convolution2D(256, 3, 3, activation='relu', name='conv3_3')(pad3_3)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2))(conv3_3)

    pad4_1 = ZeroPadding2D((1, 1))(pool3)
    conv4_1 = Convolution2D(512, 3, 3, activation='relu', name='conv4_1')(pad4_1)
    pad4_2 = ZeroPadding2D((1, 1))(conv4_1)
    conv4_2 = Convolution2D(512, 3, 3, activation='relu', name='conv4_2')(pad4_2)
    pad4_3 = ZeroPadding2D((1, 1))(conv4_2)
    conv4_3 = Convolution2D(512, 3, 3, activation='relu', name='conv4_3')(pad4_3)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2))(conv4_3)

    pad5_1 = ZeroPadding2D((1, 1))(pool4)
    conv5_1 = Convolution2D(512, 3, 3, activation='relu', name='conv5_1')(pad5_1)
    pad5_2 = ZeroPadding2D((1, 1))(conv5_1)
    conv5_2 = Convolution2D(512, 3, 3, activation='relu', name='conv5_2')(pad5_2)
    pad5_3 = ZeroPadding2D((1, 1))(conv5_2)
    conv5_3 = Convolution2D(512, 3, 3, activation='relu', name='conv5_3')(pad5_3)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2))(conv5_3)

    fc6 = Convolution2D(4096, 7, 7, activation='relu', name='fc6')(pool5)
    fc6_drop = Dropout(0.5)(fc6)
    fc7 = Convolution2D(4096, 1, 1, activation='relu', name='fc7')(fc6_drop)
    fc7_drop = Dropout(0.5)(fc7)
    fc8 = Convolution2D(2622, 1, 1, name='fc8')(fc7_drop)
    flat = Flatten()(fc8)
    out = Activation('softmax')(flat)

    model = Model(input=img, output=out)

    if weights_path:
        model.load_weights(weights_path)

    return model

In [14]:
model = vgg_face('D:\AIML- UT\Vision systems\Project 2\Part 3 - vgg_face_weights.h5')


## SVM without PCA

In [15]:
from scipy.stats import zscore
from scipy.spatial.distance import cdist
from sklearn.cluster import AgglomerativeClustering 
from scipy.spatial.distance import pdist  
from scipy.cluster.hierarchy import cophenet, dendrogram, linkage
from sklearn import svm
from sklearn.decomposition import PCA, IncrementalPCA

In [ ]:
x_train_resized=np.zeros((x_train.shape[0],50,50))
for i in range(x_train.shape[0]):
  #using cv2.resize to resize each train example to 28X28 size using Cubic interpolation
  x_train_resized[i,:,:]=cv2.resize(x_train[i],dsize=(50,50),interpolation=cv2.INTER_CUBIC)
x_train = x_train_resized.reshape(x_train_resized.shape[0], 50, 50, 1)
x_train = x_train/ 255

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.30, random_state=1)

In [16]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x]== predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [19]:
clf = svm.SVC(gamma=0.025, C=3)  

In [ ]:
clf.fit(X_train , y_train)


In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
ytest2=Y_test.to_numpy()

In [ ]:
print('Accuracy :', getAccuracy(ytest2 , y_pred))